In [17]:
import sys
print(sys.version)


3.11.9 (main, Apr  6 2024, 17:59:24) [GCC 11.4.0]


In [18]:
import pyspark
print(pyspark.__file__)


/usr/local/lib/python3.11/dist-packages/pyspark/__init__.py


In [19]:
# Define the packages to include
packages = [
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1",  # Kafka package
]

In [20]:
from pyspark.sql import SparkSession
# Initialize SparkSession

spark = SparkSession.builder \
    .appName("Hello12345") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", ",".join(packages)) \
    .getOrCreate()

In [21]:
# Subscribe to 1 topic
kafka_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka-broker-0:9092,kafka-broker-1:9092") \
  .option("subscribe", "test") \
  .option("startingOffsets", "earliest") \
  .load()


In [22]:
from pyspark.sql.functions import col, from_json

In [23]:
# Cast key and value columns to string
kafka_df = kafka_df \
    .withColumn("key", col("key").cast("string")) \
    .withColumn("value", col("value").cast("string"))


In [24]:
query = kafka_df \
    .writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName("mem_results2") \
    .start()

24/07/15 10:01:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d87cd8c8-2a4b-4c3f-8355-b3d9a4165eac. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/15 10:01:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/07/15 10:01:08 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [25]:
spark.sql('show tables').show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |mem_results2|       true|
+---------+------------+-----------+



In [26]:
spark.sql('select * from mem_results2').show()

+---+-----+-----+---------+------+--------------------+-------------+
|key|value|topic|partition|offset|           timestamp|timestampType|
+---+-----+-----+---------+------+--------------------+-------------+
|abc|  xyz| test|        0|     0|2024-07-15 09:50:...|            0|
+---+-----+-----+---------+------+--------------------+-------------+



In [16]:
spark.stop()